In [1]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [8]:
positive_samples_df = pd.read_csv("/content/Papers of Interest with Abstracts.csv")
positive_samples_df = positive_samples_df[["title", "abstract",'url']]
daily_papers_df = pd.read_csv("/content/Arxiv papers daily.csv")

In [9]:
positive_samples_df["text"] = positive_samples_df["title"] + " " + positive_samples_df["abstract"]
daily_papers_df["text"] = daily_papers_df["title"] + " " + daily_papers_df["abstract"]
positive_samples_df.head()

,title,abstract,url,text
0,Title:A Generalist Neural Algorithmic Learner,Abstract:The cornerstone of neural algorithmic...,https://arxiv.org/abs/2209.11142,Title:A Generalist Neural Algorithmic Learner ...
1,Title:Meta-Learning Fast Weight Language Models,Abstract:Dynamic evaluation of language models...,https://arxiv.org/abs/2212.02475,Title:Meta-Learning Fast Weight Language Model...
2,Title:Testing GLOM's ability to infer wholes f...,Abstract:The GLOM architecture proposed by Hin...,https://arxiv.org/abs/2211.16564,Title:Testing GLOM's ability to infer wholes f...
3,Title:Seven Sketches in Compositionality: An I...,Abstract:This book is an invitation to discove...,https://arxiv.org/abs/1803.05316,Title:Seven Sketches in Compositionality: An I...
4,Title:Evolution through Large Models,Abstract:This paper pursues the insight that l...,https://arxiv.org/abs/2206.08896,Title:Evolution through Large Models Abstract:...


In [22]:
model = SentenceTransformer('all-MiniLM-L6-v2').to('cuda')
positive_samples_embeddings = model.encode(positive_samples_df['text'].tolist(), convert_to_tensor=True)
daily_papers_embeddings = model.encode(daily_papers_df['text'].tolist(), convert_to_tensor=True)

# Calculate cosine similarities between embeddings
cosine_similarities_embeddings = cosine_similarity(daily_papers_embeddings.cpu(), positive_samples_embeddings.cpu())

# Get the most similar papers based on the embeddings
max_similarity_indices = np.argmax(cosine_similarities_embeddings, axis=1)
max_similarity_scores = np.max(cosine_similarities_embeddings, axis=1)

recommended_papers = pd.DataFrame({
    'Paper_ID': daily_papers_df['id'].values,  # Using .values to ensure correct handling
    'Title': daily_papers_df['title'].values,
    'Abstract': daily_papers_df['abstract'].values,
    'Most_Similar_Paper_ID': positive_samples_df['url'].iloc[max_similarity_indices].values,  # Added .values
    'Similarity_Score': max_similarity_scores
}, index=daily_papers_df.index)
recommended_papers.sort_values('Similarity_Score', ascending=False, inplace=True)
recommended_papers.head(5)

1173
1173
1173


,Paper_ID,Title,Abstract,Most_Similar_Paper_ID,Similarity_Score
408,http://arxiv.org/abs/2404.11483v1,"AgentKit: Flow Engineering with Graphs, not Co...",We propose an intuitive LLM prompting framewor...,https://arxiv.org/abs/2404.11483v1,0.992351
370,http://arxiv.org/abs/2404.10308v1,Hierarchical Context Merging: Better Long Cont...,Large language models (LLMs) have shown remark...,https://arxiv.org/abs/2304.11062,0.779029
14,oai:arXiv.org:2403.19888v1,MambaMixer: Efficient Selective State Space Mo...,arXiv:2403.19888v1 Announce Type: new Abstrac...,https://arxiv.org/abs/2312.00752,0.769301
524,http://arxiv.org/abs/2404.11925v1,EdgeFusion: On-Device Text-to-Image Generation,The intensive computational burden of Stable D...,https://arxiv.org/abs/2303.01469,0.754476
677,http://arxiv.org/abs/2404.11925v1,EdgeFusion: On-Device Text-to-Image Generation,The intensive computational burden of Stable D...,https://arxiv.org/abs/2303.01469,0.754476


In [23]:
positive_samples_embeddings.shape

torch.Size([313, 384])

In [27]:
import torch

class Net(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.dense1 = torch.nn.Linear(384*2,128)
    self.dense2 = torch.nn.Linear(128,32)
    self.dropout=torch.nn.Dropout(0.2)
    self.dense3 = torch.nn.Linear(32,2)

  def forward(self,x):
    x = self.dense1(x)
    x = self.dropout(x)
    x = torch.tanh(x)
    x = self.dense2(x)
    x = self.dropout(x)
    x = torch.tanh(x)
    x = self.dense3(x)
    x = self.dropout(x)
    return x

In [35]:
import torch
from torch.utils.data import DataLoader
from random import randint, random
import numpy as np

# Assuming your model is already defined as Net and moved to CUDA
model = Net().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

# Convert DataFrame texts to embeddings
# Assuming these functions are already defined and moved to cuda
# positive_samples_embeddings and daily_papers_embeddings

# Create a dataset
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, positive_samples_embeddings, daily_papers_embeddings):
        self.positive_samples_embeddings = positive_samples_embeddings
        self.daily_papers_embeddings = daily_papers_embeddings

    def __len__(self):
        # Assuming the number of daily papers is the limiting factor
        return len(self.daily_papers_embeddings)

    def __getitem__(self, idx):
        pos_sample = self.positive_samples_embeddings[randint(0, len(self.positive_samples_embeddings) - 1)]
        daily_paper = self.daily_papers_embeddings[idx]
        another_daily = self.daily_papers_embeddings[randint(0, len(self.daily_papers_embeddings) - 1)]

        if random() > 0.5:
            # 50% chance to pair with a positive sample, label is 1
            return torch.cat((pos_sample, daily_paper), dim=0), torch.tensor([1])
        else:
            # 50% chance to pair with another daily paper, label is 0
            return torch.cat((daily_paper, another_daily), dim=0), torch.tensor([0])

# Create DataLoader
dataset = MyDataset(positive_samples_embeddings, daily_papers_embeddings)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for data, labels in data_loader:
        data, labels = data.cuda(), labels.cuda()

        optimizer.zero_grad()

        outputs = model(data)
        loss = criterion(outputs, labels.squeeze(1))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(data_loader)}")

print("Training complete!")

Epoch 1/50, Loss: 0.6157299739283484
Epoch 2/50, Loss: 0.4442987305087012
Epoch 3/50, Loss: 0.38674108200782054
Epoch 4/50, Loss: 0.3500854763630274
Epoch 5/50, Loss: 0.2991571059903583
Epoch 6/50, Loss: 0.30913969673014974
Epoch 7/50, Loss: 0.2960251473091744
Epoch 8/50, Loss: 0.2933818943597175
Epoch 9/50, Loss: 0.27052644096516276
Epoch 10/50, Loss: 0.24435682776006493
Epoch 11/50, Loss: 0.19857612131415187
Epoch 12/50, Loss: 0.20150517068199209
Epoch 13/50, Loss: 0.2321770169042252
Epoch 14/50, Loss: 0.1686284667938142
Epoch 15/50, Loss: 0.2155486989665676
Epoch 16/50, Loss: 0.24081728808783195
Epoch 17/50, Loss: 0.19738252340136347
Epoch 18/50, Loss: 0.2044869368342129
Epoch 19/50, Loss: 0.2042288156981404
Epoch 20/50, Loss: 0.192197929362993
Epoch 21/50, Loss: 0.1972490300399226
Epoch 22/50, Loss: 0.16479397595331474
Epoch 23/50, Loss: 0.17632675533359116
Epoch 24/50, Loss: 0.1544790618322991
Epoch 25/50, Loss: 0.19191016508518038
Epoch 26/50, Loss: 0.17890713283339063
Epoch 27/5

In [43]:
model.eval()  # Set the model to evaluation mode

# Prepare DataLoader for all combinations
class CombinationDataset(torch.utils.data.Dataset):
    def __init__(self, daily_papers_embeddings, positive_samples_embeddings):
        self.daily_papers_embeddings = daily_papers_embeddings
        self.pmodel.eval()  # Set the model to evaluation mode

# DataLoader for evaluating every positive sample with each daily paper
class CombinationDataset(torch.utils.data.Dataset):
    def __init__(self, daily_papers_embeddings, positive_samples_embeddings):
        self.daily_papers_embeddings = daily_papers_embeddings
        self.positive_samples_embeddings = positive_samples_embeddings

    def __len__(self):
        # Total combinations
        return len(self.daily_papers_embeddings) * len(self.positive_samples_embeddings)

    def __getitem__(self, idx):
        num_pos_samples = len(self.positive_samples_embeddings)
        daily_idx = idx // num_pos_samples
        pos_idx = idx % num_pos_samples
        combined_embedding = torch.cat((self.daily_papers_embeddings[daily_idx], self.positive_samples_embeddings[pos_idx]), dim=0)
        return combined_embedding, daily_idx, pos_idx

combination_dataset = CombinationDataset(daily_papers_embeddings, positive_samples_embeddings)
combination_loader = DataLoader(combination_dataset, batch_size=1024, shuffle=False)

# Initialize arrays to track the best positive samples and their probabilities
max_probabilities = torch.full((len(daily_papers_embeddings),), -1.0, device='cuda')
best_pos_indices = torch.zeros(len(daily_papers_embeddings), dtype=torch.long, device='cuda')

with torch.no_grad():
    for embeddings, daily_indices, pos_indices in combination_loader:
        embeddings = embeddings.cuda()
        outputs = model(embeddings)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)[:, 1]  # Probabilities for label '1'

        for i in range(len(probabilities)):
            daily_idx = daily_indices[i].item()
            pos_idx = pos_indices[i].item()
            prob = probabilities[i].item()
            if prob > max_probabilities[daily_idx]:
                max_probabilities[daily_idx] = prob
                best_pos_indices[daily_idx] = pos_idx

              #  print(daily_idx,pos_idx,prob)

# Convert indices and probabilities to CPU for DataFrame creation
best_pos_indices = best_pos_indices.cpu()
max_probabilities = max_probabilities.cpu()

recommended_papers = pd.DataFrame({
    'Paper_ID': daily_papers_df['id'].values,
    'Title': daily_papers_df['title'].values,
    'Abstract': daily_papers_df['abstract'].values,
    'Most_Similar_Paper_ID': positive_samples_df['url'].iloc[best_pos_indices].values,
    'Similarity_Score': max_probabilities.numpy()
}, index=daily_papers_df.index)
recommended_papers.sort_values('Similarity_Score', ascending=False, inplace=True)
recommended_papers.head(5)

,Paper_ID,Title,Abstract,Most_Similar_Paper_ID,Similarity_Score
733,http://arxiv.org/abs/2404.12803v1,TextSquare: Scaling up Text-Centric Visual Ins...,Text-centric visual question answering (VQA) h...,https://arxiv.org/abs/2312.03173,0.999340
1119,http://arxiv.org/abs/2404.17563v1,An exactly solvable model for emergence and sc...,Deep learning models can exhibit what appears ...,https://arxiv.org/abs/2312.03173,0.999044
726,http://arxiv.org/abs/2404.12843v1,Towards Logically Consistent Language Models v...,Large language models (LLMs) are a promising v...,https://arxiv.org/abs/2312.03173,0.998964
873,http://arxiv.org/abs/2404.14994v1,Transformers Can Represent $n$-gram Language M...,Plenty of existing work has analyzed the abili...,https://arxiv.org/abs/2312.03173,0.998790
770,http://arxiv.org/abs/2404.12596v1,Parameter Efficient Diverse Paraphrase Generat...,"Over the past year, the field of Natural Langu...",https://arxiv.org/abs/2312.03173,0.998704
